# Connect to EMAP star

In [ ]:
from datetime import datetime, timedelta
import os
from pathlib import Path
from pprint import pprint
import urllib

import arrow
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

EMAP credentials are allocated per user and not stored in the environment variables.
You do not want your credentials to leak into the source repository.

One way of safeguarding is to create a file called `secret` at the top level of your repository.   
Do this here in Jupyter and not a local copy of the repo.  

The first line should be your UDS **username** and the second line should be your UDS **password**.

`secret` has been added to `.gitignore` and will be excluded from the repository.

In [ ]:
os.environ['EMAP_DB_USER'], os.environ['EMAP_DB_PASSWORD'] = Path('../../secret').read_text().strip().split('\n')

In [ ]:
uds_host = os.getenv('EMAP_DB_HOST')
uds_name = os.getenv('EMAP_DB_NAME')
uds_port = os.getenv('EMAP_DB_PORT')
uds_user = os.getenv('EMAP_DB_USER')
uds_passwd = os.getenv('EMAP_DB_PASSWORD')

In [ ]:
emapdb_engine = create_engine(f'postgresql://{uds_user}:{uds_passwd}@{uds_host}:{uds_port}/{uds_name}')

In [ ]:
vitals_df = pd.read_sql(
    """
-- Example script showing how to work with observations

-- V simple view that finds recent observations 
-- for current inpatients in the last few minutes


SELECT
  -- observation details
   ob.visit_observation_id
  ,ob.hospital_visit_id
  ,ob.observation_datetime

  --,ob.visit_observation_type_id
  --,ot.id_in_application

  -- label nicely
  ,CASE 
    WHEN ot.id_in_application = '10' THEN 'SpO2'
    WHEN ot.id_in_application = '5' THEN 'BP'
    WHEN ot.id_in_application = '3040109304' THEN 'Oxygen'
    WHEN ot.id_in_application = '6' THEN 'Temp'
    WHEN ot.id_in_application = '8' THEN 'Pulse'
    WHEN ot.id_in_application = '9' THEN 'Resp'
    WHEN ot.id_in_application = '6466' THEN 'AVPU'

  END AS vital

  ,ob.value_as_real
  ,ob.value_as_text
  ,ob.unit 
  
FROM
  star.visit_observation ob
-- observation look-up
LEFT JOIN
  star.visit_observation_type ot
  on ob.visit_observation_type_id = ot.visit_observation_type_id

WHERE
ob.observation_datetime > NOW() - '5 MINS'::INTERVAL	
AND
ot.id_in_application in 

  (
  '10'            --'SpO2'                  -- 602063230
  ,'5'            --'BP'                    -- 602063234
  ,'3040109304'   --'Room Air or Oxygen'    -- 602063268
  ,'6'            --'Temp'                  -- 602063248
  ,'8'            --'Pulse'                 -- 602063237
  ,'9'            --'Resp'                  -- 602063257
  ,'6466'         -- Level of consciousness
)
ORDER BY ob.observation_datetime DESC
;
    """,
    emapdb_engine
)

In [ ]:
vitals_df